## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

A seguir, um exemplo desta classificação com os dados `single target` de `AVI dataset`:

In [96]:
import mne
import numpy as np

# tarefa 1
data = mne.read_epochs("../../datasets/beta/beta-epo.fif")

Reading c:\Users\vinny\OneDrive\Documentos\UTFPR\Reconhecimento de Padroes\tutoriais\SSVEP\src\beta\..\..\datasets\beta\beta-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2996.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


In [97]:
# TAREFA 2


# entendimento dos dados
print(data.get_data().shape)
data

(160, 64, 750)


Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.6: 412.8: 413.0: 413.2: 413.4: 413.6: 413.8: 414.0: 414.2: 414.4: 414.6: 414.8: 415.0: 415.2: 415.4: 415.6: 415.8: 48.0: 48.2: 48.4: 48.6: 48.8: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 2.996 s
Baseline,off


#### Lista apenas para orientação do nome dos canais

Fp1: eletrodo fronto-polar esquerdo

Fp2: eletrodo fronto-polar direito

F3: eletrodo frontal esquerdo

F4: eletrodo frontal direito

Fpz: eletrodo fronto-polar da linha média

Fz: eletrodo frontal da linha há média

C3: eletrodo central esquerdo

C4: eletrodo central direito

Cz: eletrodo central da linha média

P3: eletrodo parietal esquerdo

P4: eletrodo parietal direito

Pz: eletrodo parietal da linha média

O1: eletrodo occipital esquerdo

O2: eletrodo occipital direito

Oz: eletrodo occipital da linha média

F7: eletrodo temporal anterior esquerdo

F8: eletrodo temporal anterior direito

T3: eletrodo temporal médio esquerdo

T4: eletrodo temporal médio direito

T5: eletrodo temporal posterior esquerdo

T6: eletrodo temporal posterior direito.


In [98]:
channels = list(np.load("../../datasets/beta/channels_beta.npy"))
print(channels)

['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'M1', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'M2', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']


In [99]:
# TAREFA 3
import matplotlib.pyplot as plt

data = mne.read_epochs("../../datasets/beta/beta-epo.fif")
channelList = ['T7', 'T8', 'O1', 'O2', 'OZ', 'FZ', 'PZ' ]
rest = [item for item in channels if item not in channelList]

data.drop_channels(rest)
print(data.get_data().shape)


targets = [float(item) for item in data.event_id.keys()]
print("Possíveis alvos:", targets)

hits = 0
threshold = 0.35

for i in range(len(data)):
    dict_y = {str(value): index  for index, value in enumerate(targets)}

    for target in targets:
        fmin = target - threshold
        fmax = target + threshold
        sample = (data[i].compute_psd(method='multitaper', fmin=fmin,
                                          fmax=fmax, verbose=False))
        values = np.array((sample)[0][0])

        values_avg = np.median(values)
        dict_y[str(target)] = values_avg

    # features.append(sample)
    target_frequency = max(dict_y, key=dict_y.get)
    if float(target_frequency) == float(list(data[i].event_id)[0]):
        hits += 1

# X = np.ndarray(features)
# print("Formato dos dados calculados:", X.shape)

Reading c:\Users\vinny\OneDrive\Documentos\UTFPR\Reconhecimento de Padroes\tutoriais\SSVEP\src\beta\..\..\datasets\beta\beta-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2996.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
(160, 7, 750)
Possíveis alvos: [8.0, 8.2, 8.4, 8.6, 8.8, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6, 11.8, 12.0, 12.2, 12.4, 12.6, 12.8, 13.0, 13.2, 13.4, 13.6, 13.8, 14.0, 14.2, 14.4, 14.6, 14.8, 15.0, 15.2, 15.4, 15.6, 15.8]


In [100]:
labels = np.load("../../datasets/beta/labels_beta.npy")

acc = 100 * hits / len(labels)
print(f'\nPorcentagem de acerto: {acc:.2f}%')
# hits, len(labels)


Porcentagem de acerto: 17.50%
